# CEE 598: Uncertainty Quantification
## Assignment Two
### Noah Garfinkle (garfink2@illinois edu)
### 13 February 2020

# Problem 1

Using inverse transform sampling, write a code that can be used to generate n samples from the Cauchy distribution.  The Cauchy PDF and CDF are given by

$$
f_X\left(x\right)=\frac{1}{\pi\gamma\left[1+\left(\frac{x-x_0}{\gamma}\right)^2\right]}
$$

$$
F_X\left(x\right)=\frac{1}{\pi}\arctan{\left(\frac{x-x_0}{\gamma}\right)}+\frac{1}{2}
$$

Where $x_o$ is the median, and $\gamma$ is a scale factor.  The inputs to your code should be the number of samples $n$, and the numerical values for $x_0$ and $\gamma$.  Compute the generated samples with the target PDF, for a particular choice of $x_0$ and $\gamma$ (you’re free to choose any value).  You can use histogram plots or any available kernel density estimator codes.  Discuss your results.

# Problem 2

You are asked to provide a random number generator from the Gamma PDF, $Gamma(x;2,2)$, given by

$$
f_X\left(x\right)=\frac{1}{4}xe^{-\frac{x}{2}}
$$

You are supposed to use rejection sampling, and choose the envelope PDF, i.e. $cg(x)$, to be a scaled version of Cauchy PDF (given in Problem 1).

## Problem 2.1

Show how you modify your Cauchy PDF to become an upper bound for the Gamma PDF everywhere.  The parameters that you can vary so that this upper bound condition is satisfied are $x_0$, $\gamma$, and also $c$.  There are several combinations of these parameters that can make the Cauchy PDF an envelope distribution.  Discussion how some of these conditions can be more favorable than others.

## Problem 2.2

Write an acceptance-rejection sampling code that can be used to generate $n$ samples from the given Gamma PDF.  Use the code you wrote for Problem 1 to draw samples from $g_X(x)$ within your rejection sampling algorithm.


## Problem 2.3

Compare the generated samples with the targeted PDF.  You can use histogram plots or available kernel density estimator codes.  Discuss your results.
